## Variance partitioning analysis

Dec-16-2021 by Guo Jiahui

This script contains functions and examples to run the variance partitioning analysis.

This script runs an example analysis for the results shown in Figure 5D & 5E.

In [1]:
import numpy as np
from scipy.spatial.distance import squareform
import read_data as rd

In [2]:
%load_ext rpy2.ipython

In [3]:
def variance_partitioning_behavioral(rdm_face_run, rdm_obj_run, behav_rdm_run):
    %Rpush rdm_face_run rdm_obj_run behav_rdm_run
    %R library(vegan)

    %R rda.all <- rda (behav_rdm_run ~ rdm_face_run + rdm_obj_run)
    %R rda.face <- rda (behav_rdm_run ~ rdm_face_run)
    %R rda.obj <- rda (behav_rdm_run ~ rdm_obj_run)
    %R abc <- RsquareAdj (rda.all)$adj.r.squared
    %R ab <- RsquareAdj (rda.face)$adj.r.squared
    %R bc <- RsquareAdj (rda.obj)$adj.r.squared
    %R b = ab + bc - abc
    %R a = ab - b
    %R c = bc - b
    %R -o a,b,c,abc

    aa1 = np.array(a)[0]
    bb1 = np.array(b)[0]
    cc1 = np.array(c)[0]
    abc1 = np.array(abc)[0]
    return aa1, bb1, cc1, abc1

In [4]:
def variance_partitioning_neural(rdm_face_run, rdm_obj_run, behav_rdm_run, neural_rdm_run):
    %Rpush rdm_face_run rdm_obj_run behav_rdm_run neural_rdm_run
    %R library(vegan)

    %R rda.all <- rda (neural_rdm_run ~ rdm_face_run + rdm_obj_run + behav_rdm_run)
    %R rda.face <- rda (neural_rdm_run ~ rdm_face_run)
    %R rda.obj <- rda (neural_rdm_run ~ rdm_obj_run)
    %R rda.behav <- rda (neural_rdm_run ~ behav_rdm_run)
    %R rda.face_obj <- rda (neural_rdm_run ~ rdm_face_run + rdm_obj_run)
    %R rda.obj_behav <- rda (neural_rdm_run ~ rdm_obj_run + behav_rdm_run)
    %R rda.behav_face <- rda (neural_rdm_run ~ behav_rdm_run + rdm_face_run)
    %R abcdefg <- RsquareAdj (rda.all)$adj.r.squared
    %R adfg <- RsquareAdj (rda.face)$adj.r.squared
    %R bdeg <- RsquareAdj (rda.obj)$adj.r.squared
    %R cefg <- RsquareAdj (rda.behav)$adj.r.squared
    %R abdefg <- RsquareAdj (rda.face_obj)$adj.r.squared
    %R bcdefg <- RsquareAdj (rda.obj_behav)$adj.r.squared
    %R acdefg <- RsquareAdj (rda.behav_face)$adj.r.squared
    %R a = abcdefg - bcdefg
    %R b = abcdefg - acdefg
    %R c = abcdefg - abdefg
    %R d = abcdefg - cefg - a - b
    %R e = abcdefg - adfg - b - c
    %R f = abcdefg - bdeg - a - c
    %R g = adfg - a - d - f
    %R -o a,b,c,d,e,f,g,abcdefg

    a1 = np.array(a)[0]
    b1 = np.array(b)[0]
    c1 = np.array(c)[0]
    d1 = np.array(d)[0]
    e1 = np.array(e)[0]
    f1 = np.array(f)[0]
    g1 = np.array(g)[0]
    abcdefg1 = np.array(abcdefg)[0]
    return a1, b1, c1, d1, e1, f1, g1, abcdefg1

In [5]:
def prepare_dcnn_rdms_run(face_arcface='fc1', face_alexnet='fc2', face_vgg16='fc2', object_alexnet='fc2', object_vgg16='fc2', run=0):
    rdm_face_arcface = rd.get_dcnn_rdm(f'{data_dir}/DCNN_RDMs', 'Face_ArcFace', face_arcface)
    rdm_face_alexnet = rd.get_dcnn_rdm(f'{data_dir}/DCNN_RDMs', 'Face_AlexNet', face_alexnet)
    rdm_face_vgg16 = rd.get_dcnn_rdm(f'{data_dir}/DCNN_RDMs', 'Face_VGG16', face_vgg16)
    rdm_object_alexnet = rd.get_dcnn_rdm(f'{data_dir}/DCNN_RDMs', 'Object_AlexNet', object_alexnet)
    rdm_object_vgg16 = rd.get_dcnn_rdm(f'{data_dir}/DCNN_RDMs', 'Object_VGG16', object_vgg16)

    rdm_face = np.mean([rdm_face_arcface, rdm_face_alexnet, rdm_face_vgg16], axis=0)
    rdm_obj = np.mean([rdm_object_alexnet, rdm_object_vgg16], axis=0)

    rdm_obj_run = rd.get_single_run_rdm(rdm_obj, run=run)
    rdm_face_run = rd.get_single_run_rdm(rdm_face, run=run)
    
    return rdm_face_run, rdm_obj_run

In [6]:
## Below is example code to run the variance paritioning analysis to explain the variance 
## of an example neural ROI representational geometry, using the mean RDMs of the final layers of face-, object-DCNN, as well as the behavioral RDM.
## Analysis is carried out run-wise, and results were avaraged across runs.
## This analysis is an example of the analyses done for Figure 5E, lower panel. 

In [7]:
data_dir = '../data'

## roi: 'raFFA', 'laFFA'
roi = 'raFFA'

In [8]:
aas = []
bbs = []
ccs = []
dds = []
ees = []
ffs = []
ggs = []
abcdefgs = []
for run in range(12):
    neural_rdm_run = rd.get_neural_rdm(f'{data_dir}/neural_RDMs', roi, run=run, mean=True)
    behav_rdm_run = rd.get_behavioral_rdm(f'{data_dir}/behavioral_RDMs', run=run, mean=True)
    rdm_face_run, rdm_obj_run = prepare_dcnn_rdms_run(face_arcface='fc1', face_alexnet='fc2', face_vgg16='fc2', object_alexnet='fc2', object_vgg16='fc2', run=run)
    a1, b1, c1, d1, e1, f1, g1, abcdefg1 = variance_partitioning_neural(rdm_face_run, rdm_obj_run, behav_rdm_run, neural_rdm_run)
    aas.append(a1)
    bbs.append(b1)
    ccs.append(c1)
    dds.append(d1)
    ees.append(e1)
    ffs.append(f1)
    ggs.append(g1)
    abcdefgs.append(abcdefg1)

m_aas = np.mean(aas)
m_bbs = np.mean(bbs)
m_ccs = np.mean(ccs)
m_dds = np.mean(dds)
m_ees = np.mean(ees)
m_ffs = np.mean(ffs)
m_ggs = np.mean(ggs)
m_abcdefgs = np.mean(abcdefgs)

R[write to console]: Loading required package: permute

R[write to console]: Loading required package: lattice

R[write to console]: This is vegan 2.5-7



In [9]:
total_squared = '{:.2f}'.format(m_abcdefgs*100)
ap = '{:.1f}'.format((m_aas/m_abcdefgs)*100)
bp = '{:.1f}'.format((m_bbs/m_abcdefgs)*100)
cp = '{:.1f}'.format((m_ccs/m_abcdefgs)*100)
abp = '{:.1f}'.format((m_dds/m_abcdefgs)*100)
acp = '{:.1f}'.format((m_ffs/m_abcdefgs)*100)
bcp = '{:.1f}'.format((m_ees/m_abcdefgs)*100)
abcp = '{:.1f}'.format((m_ggs/m_abcdefgs)*100)

In [10]:
print(f'--- Variance Partitioning: Neural ROI {roi}, DCNN final layers ---')
print(f'Total r-squared: {total_squared}%')
print(f'a: {ap}%')
print(f'b: {bp}%')
print(f'c: {cp}%')
print(f'ab: {abp}%')
print(f'ac: {acp}%')
print(f'bc: {bcp}%')
print(f'abc: {abcp}%')

--- Variance Partitioning: Neural ROI raFFA, DCNN final layers ---
Total r-squared: 0.36%
a: 21.2%
b: 20.9%
c: 50.8%
ab: -0.4%
ac: 0.0%
bc: 6.7%
abc: 0.9%


In [11]:
## Below is a peiece example code to run the variance paritioning analysis to explain the variance of the behavioral representational geometry,
## using the mean RDMs of the final layers of face-, object-DCNN.
## Analysis is carried out run-wise, and results were avaraged across runs.
## This analysis is outputs results for plotting Figure 5D, lower panel. 

In [12]:
aas = []
bbs = []
ccs = []
abcs = []
for run in range(12):
    neural_rdm_run = rd.get_neural_rdm(f'{data_dir}/neural_RDMs', roi, run=run, mean=True)
    behav_rdm_run = rd.get_behavioral_rdm(f'{data_dir}/behavioral_RDMs', run=run, mean=True)
    rdm_face_run, rdm_obj_run = prepare_dcnn_rdms_run(face_arcface='fc1', face_alexnet='fc2', face_vgg16='fc2', object_alexnet='fc2', object_vgg16='fc2', run=run)
    a1, b1, c1, abc1 = variance_partitioning_behavioral(rdm_face_run, rdm_obj_run, behav_rdm_run)
    aas.append(a1)
    bbs.append(b1)
    ccs.append(c1)
    abcs.append(abc1)

m_aas = np.mean(aas)
m_bbs = np.mean(bbs)
m_ccs = np.mean(ccs)
m_abcs = np.mean(abcs)

In [13]:
total_squared = '{:.2f}'.format(m_abcs*100)
ap = '{:.1f}'.format((m_aas/m_abcs)*100)
bp = '{:.1f}'.format((m_ccs/m_abcs)*100)
abp = '{:.1f}'.format((m_bbs/m_abcs)*100)

In [14]:
print(f'--- Variance Partitioning: Behavioral RDM, DCNN final layers ---')
print(f'Total r-squared: {total_squared}%')
print(f'a: {ap}%')
print(f'b: {bp}%')
print(f'ab: {abp}%')

--- Variance Partitioning: Behavioral RDM, DCNN final layers ---
Total r-squared: 5.37%
a: 23.6%
b: 70.9%
ab: 5.5%
